In [ ]:
from dotenv import load_dotenv
import os
import openai

# Charger les variables d'environnement depuis un fichier .env
load_dotenv()

# Récupérer la clé API directement depuis les variables d'environnement
openai.api_key = os.getenv("OPENAI_API_KEY")
from openai import OpenAI
import pandas as pd 
from utils import recommender
import geopy
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import certifi
import ssl 
from geopy.exc import GeocoderTimedOut
import time
import numpy as np
import json

from unidecode import unidecode

client = OpenAI()

In [ ]:

# Liste des questions
questions = [
    "Avez-vous des Clio 4 1.5 DCI en occasion ?",
    "Quels sont les véhicules automatiques disponibles ?",
    "Avez-vous un Rifter disponible ?",
    "Quels sont vos modèles hybrides ?",
    "Les véhicules électriques en occasion sont-ils disponibles ?",
    "Puis-je réserver une Citroën C3 ?",
    "Est-ce que ce véhicule (modèle précis) est encore disponible ?",
    "Quels véhicules proposez-vous pour 10 000 € ?",
    "Quels sont vos modèles hybrides rechargeables disponibles ?",
    "Quels véhicules automatiques sont disponibles pour 15 000 € ?",
    "Pouvez-vous confirmer la disponibilité de ce SUV ?",
    "Quels sont les SUV disponibles pour 20 000 € ?",
    "Quels modèles tout-terrain avez-vous en stock ?",
    "Avez-vous des modèles électriques d'occasion ?",
    "Proposez-vous des véhicules de moins de 5 ans avec garantie ?",
    "Quels modèles utilitaires sont disponibles à la location ?",
    "Quels modèles compacts automatiques avez-vous en stock ?",
    "Avez-vous des Renault Zoé d'occasion en stock ?",
    "Quels modèles de citadines automatiques avez-vous ?",
    "Quels modèles sont compatibles avec les familles nombreuses ?",
    "Quels véhicules récents sont disponibles à moins de 20 000 € ?",
    "Quels modèles 4x4 avez-vous d'occasion ?",
    "Quels utilitaires avec portes latérales sont disponibles ?",
    "Quels SUV sont disponibles en version hybride rechargeable ?"
]

# Fonction pour transformer la question en critères via GPT
def generate_criteria_from_question(question):
    response = client.chat.completions.create(
        messages=[{
            "role": "system",
            "content": (
                f"Réponds à la description suivante avec un JSON comportant les critères nécessaires pour "
                f"rechercher un véhicule : 'brand', 'model', 'energy', 'gearbox' "
                f"energy peut etre 'Essence', 'Diesel', 'Hybride' ou 'Electrique'. "
                f"gearbox peut etre 'Manuelle' ou 'Automatique'. "
                f"n'ajoute pas de critères si ils ne sont pas mentionnés dans la question. "

                f"Description : {question}"
            )
        }],
        model="gpt-4o-mini",
        response_format={"type": "json_object"}
    )
    return json.loads(response.choices[0].message.content)

# Fonction de test
def test_recommender():
    for question in questions:
        print(f"\nTesting question: {question}")
        
        # Étape 1 : Obtenir les critères
        try:
            criteria = generate_criteria_from_question(question)
        except Exception as e:
            print(f"Erreur lors de la génération des critères pour '{question}': {e}")
            continue
        
        # Étape 2 : Passer les critères à la fonction recommender
        try:
            zip_code = "75001"  # Exemple de code postal
            result = recommender(
                full_description=question,
                zip_code=zip_code,
                brand=criteria.get("brand"),
                model=criteria.get("model"),
                version=criteria.get("version"),
                energy=criteria.get("energy"),
                max_price=criteria.get("max_price")
            )
            
            # Étape 3 : Afficher les résultats
            print("Résultats :")
            print(result)
        
        except Exception as e:
            print(f"Erreur lors de l'exécution de 'recommender' pour '{question}': {e}")

# Exécuter les tests
test_recommender()
